# Features as Tools 

This tutorial guides you through creating an LLM generated restaurant recommendation function by using Features as Tools.

This is an example of how an LLM can make use of feature views as data retrieval tools. At development time, you provide the set of feature views that the LLM agent is allowed to call. The LLM uses the feature view's description field to determine what tools are relevant for answering the user question. It  makes the corresponing feature retrieval call(s) using the Tecton entity keys provided in the context of the call.


## Install Packages

In [2]:
!pip install 'tecton[rift]==0.10.0b32' gcsfs s3fs --force-reinstall
!pip install "$HOME/Downloads/tecton_gen_ai-0.0.4-py3-none-any.whl" --force-reinstall

!pip install openai
!pip install langchain-openai
!pip install langchain
!pip install langchain_community
!pip install langchain_core
!pip install llama-index
!pip install llama-index-llms-openai

  Using cached tecton-0.10.0b32-py3-none-any.whl.metadata (6.8 kB)
  Using cached gcsfs-2024.6.1-py2.py3-none-any.whl.metadata (1.6 kB)
  Using cached s3fs-2024.6.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
  Using cached pendulum-2.1.2-cp39-cp39-macosx_14_0_arm64.whl
  Using cached protobuf-5.27.3-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached pytimeparse-1.1.8-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached pandas-2.2.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached texttable-1.7.0-py2.py3-none-any.whl.metadata (9.8 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.

# Features as Tools

In the following cell you'll create a Tecton Agent with a simple prompt and two different feature views for the LLM as tools.
The system prompt in this example, instructs the LLM to address the user by name and to only suggest restaurants that in near the location. In order to address the use by name the LLM will need to use a tools.
The location is configured to be provided as part of the context in the request by using a RequestSource object. 

In [5]:
from tecton_gen_ai.agent import AgentService, AgentClient
from tecton_gen_ai import prompt
from tecton_gen_ai.core_utils import make_request_source

def restaurant_recommender_agent( user_info_fv, recent_visit_fv):    

    location_request = make_request_source(location = str)
    
    @prompt(sources=[ location_request])
    def sys_prompt(location_request ):
        location = location_request["location"]
        return f"""
        Address the user by name.
        You are a consierge service that recommends restaurants.
        Respond to the user's question. 
        If the user asks for a restaurant recommendation respond with a specific restaurant that you know and suggested menu items. 
        Only suggest restaurants that are in or near {location}. 
        """
            
    return AgentService(
        name="restaurant_recommender",
        prompts=[sys_prompt],
        tools=[ user_info_fv, recent_visit_fv],  # feature views used as tools
    )

The `sys_prompt` instructions above require the LLM to "address the user by name" which requires the use of a tool.
 
In the next cell we create a couple of mock feature views that provide basic user information and the list of their recently visited restaurants respectively.

In practice, these feature views would be implemented as stream feature views that would provide the latest user information and recent visits within seconds of the corresponding user activity.

In [6]:
import pandas as pd
from tecton import RequestSource
from tecton.types import Field, String


from tecton_gen_ai.testing import mock_batch_feature_view


# mock user info data
user_info = pd.DataFrame(
        [
            {"user_id": "user1", "name": "Jim",  "age": 30, "food_preference": "American"},
            {"user_id": "user2", "name": "John", "age": 40, "food_preference": "Italian"},
            {"user_id": "user3", "name": "Jane", "age": 50, "food_preference": "Chinese"},
        ]
    )
user_info_fv = mock_batch_feature_view( "user_info_fv", user_info, entity_keys=["user_id"], description="User's basic information.")

# mock user's recent visits 
recent_eats = pd.DataFrame(
        [
            {"user_id": "user1", "last_3_visits":str(["Mama Ricotta's", "The Capital Grille", "Firebirds Wood Fired Grill"])},
            {"user_id": "user2", "last_3_visits":str(["Mama Ricotta's", "Villa Antonio", "Viva Chicken"])},
            {"user_id": "user3", "last_3_visits":str(["Wan Fu", "Wan Fu Quality Chinese Cuisine", "Ru San's"])},
        ]
    )
recent_eats_fv = mock_batch_feature_view( "recent_eats_fv", recent_eats, entity_keys=["user_id"], description="User's recent restaurant visits.")

The feature views entity key in both cases is `user_id`, the key will need to be provided in the context when invoking the LLM agent. 

In the following cell, you create a Tecton AgentClient with uses the mock feature views defined above.

In [7]:
# create the Tecton Agent
recommender_agent = restaurant_recommender_agent( user_info_fv, recent_eats_fv )

# create a client to invoke the agent
client = AgentClient.from_local( recommender_agent )

## Put it all together

The Tecton AgentClient created above can be used to create a LangChain or LlamaIndex runnable agent.

In the cell below you will instantiate an LLM model using OpenAI's `gpt-4o-mini` model and create a LangChain agent that is ready to use the tools we've provided above. In the following section, you'll do the same using a LlamaIndex.

Obtain an [OpenAI API key](https://platform.openai.com/api-keys) and replace "your-openai-key" in the following cell.

In [9]:
import openai as oa
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# replace with your key
os.environ["OPENAI_API_KEY"] = "your-openai-key"

# instantiate LLM model for  LangChain 
langchain_llm = ChatOpenAI(model="gpt-4o-mini")

# create invokable agent for LangChain 
langchain_agent = client.make_agent(langchain_llm, system_prompt="sys_prompt")


### Testing with different context

In the following cells you'll provide different context and invoke the LLM agent to test how features as tools provide user specifc personalization. 

The first test uses the `langchain_agent`, it's input and output are a dictionaries with "input" and "output" keys:

In [10]:
print("Context - user 1 in Charlotte \n\nResponse:")
with client.set_context({"user_id": "user1", "location":"Ballantyne, Charlotte, NC"}):
    print(langchain_agent.invoke({"input":"what do you know about me and what would you recommend"})["output"])

Context - user 1 in Charlotte 

Response:
Hello Jim! Based on your recent visits, it seems you enjoy a variety of American cuisine. I recommend trying **The Palm** in Ballantyne. They offer a fantastic menu with options like their famous **Filet Mignon** and **Lobster Tail**. For a lighter option, the **House Salad** is also a great choice. Enjoy your meal!


## Different Context for Different Users

In the following cells you can see how the response changes based on the `user_id` and the `location` provided resulting in a personalized response for each user and based on their current location.

The following test uses the `llamaindex_agent` with its `chat` method that receiving a query string as input and returns output as a string as well:

In [11]:
print( "Context - user 2 in Charlotte \n\nResponse:")
with client.set_context({"user_id": "user2", "location":"Ballantyne, Charlotte, NC"}):
    print(langchain_agent.invoke({"input":"I want to try a place tonight that I've never been to that also fits my preference"})["output"])


Context - user 2 in Charlotte 

Response:
Hi John! Since you're looking for an Italian restaurant that you haven't tried yet, I recommend checking out **Maggiano's Little Italy** in Ballantyne. 

Their menu features delicious options like:

- **Bruschetta** - Toasted bread topped with a mix of fresh tomatoes, basil, and mozzarella.
- **Fettuccine Alfredo** - A classic dish with creamy Alfredo sauce, perfect for pasta lovers.
- **Chicken Parmigiana** - Breaded chicken breast topped with marinara sauce and melted mozzarella.

Enjoy your dinner!


In [12]:
print( "Context - user 3 in Charlotte \n\nResponse:")
with client.set_context({"user_id": "user3", "location":"Charlotte, NC"}):
    print(langchain_agent.invoke({"input":"Recommend one of my regular spots for dinner"})["output"])


Context - user 3 in Charlotte 

Response:
Hi Jane! Since you enjoy Chinese cuisine and have visited Wan Fu and Ru San's recently, I recommend you try **Wan Fu Quality Chinese Cuisine** again. 

Their menu features delicious options like:

- **Peking Duck**: A classic dish with crispy skin and tender meat.
- **Kung Pao Chicken**: A spicy stir-fry with chicken, peanuts, and vegetables.
- **Mongolian Beef**: Tender beef with scallions in a savory sauce.

Enjoy your dinner!


### With LlamaIndex 

Tecton also integrates with LlamaIndex in the same way it does with LangChain in order to provide enriched prompts and features as tools.

The only difference is in the LangChain`.invoke` vs LlamaIndex`.chat` methods and their signatures. Tecton delivers fresh context in the same way for both:

In [13]:
from llama_index.llms.openai import OpenAI

# instantiate LLM model for with LlamaIndex integration 
llamaindex_llm = OpenAI(model="gpt-4o-mini")

# create invokable agent for LlamaIndex 
llamaindex_agent = client.make_agent(llamaindex_llm, system_prompt="sys_prompt")

print( "Context - user 3 in Charlotte \n\nResponse:")
with client.set_context({"user_id": "user3", "location":"Charlotte, NC"}):
    print(llamaindex_agent.chat("Recommend one of my regular spots for dinner"))


Context - user 3 in Charlotte 

Response:
It looks like you've recently visited Wan Fu and Ru San's. If you're in the mood for some delicious sushi, I recommend heading back to Ru San's. Their sushi rolls are fantastic! You might want to try the "Dynamite Roll" or the "Rainbow Roll" for a delightful dinner experience. Enjoy!


Notice that in the LlamaIndex case, given that it is stateful, a new instance of the agent is needed if the context is different between calls.

In [14]:
# create new instance of agent for LlamaIndex 
llamaindex_agent = client.make_agent(llamaindex_llm, system_prompt="sys_prompt")

print( "Context - user 2 in New York \n\nResponse:")
with client.set_context({"user_id": "user2", "location":"New York, NY"}):
    print(llamaindex_agent.chat("something romantic for dinner"))


Context - user 2 in New York 

Response:
For a romantic dinner, I recommend **Carbone**, an upscale Italian restaurant located in Greenwich Village. The ambiance is intimate and perfect for a special evening.

Here are some suggested menu items:
- **Spicy Rigatoni Vodka**: A signature dish that's a must-try.
- **Veal Parmesan**: Tender and flavorful, it's a classic choice.
- **Tiramisu**: End your meal on a sweet note with this delightful dessert.

Make sure to make a reservation, as it can get quite busy! Enjoy your evening, John!


# Conclusion

Tecton delivers real-time, streaming and batch feature pipelines in production. Enhancing an LLMs context through features as tools, brings the power of fresh context to generative AI applications. By using features as tools for the LLM, you give the LLM the ability to retrieve relevant information only when the user question needs requires it.